In [ ]:
# this cell is only for colab execution
from google.colab import drive
drive.mount('/content/drive')

In [1]:
# this cell is only for colab execution
%cd ../content/drive/MyDrive/SymbolicMathematics-master/

[Errno 2] No such file or directory: '../content/drive/MyDrive/SymbolicMathematics-master/'
/content


In [ ]:
import os
import numpy as np
import sympy as sp
import torch

from src.utils import AttrDict
from src.envs import build_env
from src.model import build_modules

from src.utils import to_cuda
from src.envs.sympy_utils import simplify

In [ ]:
t = sp.symbols('t')

C,D,E = t+2,1+0*t,t**3+sp.sin(t)

In [ ]:
#[C(t),D(t),E(t)]

display(C,D,E)

t + 2

1

t**3 + sin(t)

In [ ]:
# trained model

model_path_1 = 'ode1.pth'
assert os.path.isfile(model_path_1)

In [ ]:
params_1 = params_1 = AttrDict({

    # environment parameters
    'env_name': 'char_sp',
    'int_base': 10,
    'balanced': False,
    'positive': True,
    'precision': 10,
    'n_variables': 1,
    'n_coefficients': 1,
    'leaf_probs': '0.6,0.2,0.2,0',
    'max_len': 512,
    'max_int': 5,
    'max_ops': 15,
    'max_ops_G': 15,
    'clean_prefix_expr': True,
    'rewrite_functions': '',
    'tasks': 'ode1',
    'operators': 'add:10,sub:3,mul:10,div:5,sqrt:4,pow2:4,pow3:2,pow4:1,pow5:1,ln:4,exp:4,sin:4,cos:4,tan:4,asin:1,acos:1,atan:1,sinh:1,cosh:1,tanh:1,asinh:1,acosh:1,atanh:1',

    # model parameters
    'cpu': False,
    'emb_dim': 1024,
    'n_enc_layers': 6,
    'n_dec_layers': 6,
    'n_heads': 8,
    'dropout': 0,
    'attention_dropout': 0,
    'sinusoidal_embeddings': False,
    'share_inout_emb': True,
    'reload_model': './ode1.pth',    

})

In [ ]:
# building the enviroment
env_1 = build_env(params_1)

In [ ]:
modules_1 = build_modules(env_1, params_1)
encoder_1 = modules_1['encoder']
decoder_1 = modules_1['decoder']

In [ ]:
x = env_1.local_dict['x']
f = env_1.local_dict['f']
a8 = env_1.local_dict['a8']

In [ ]:
D_1=D.subs(t,x)

In [ ]:
# this is an easy ODE problem just for testing
ode = sp.Derivative(f(x))-D_1*f(x)

display(ode)

-f(x) + Derivative(f(x), x)

In [ ]:
ode_prefix = env_1.sympy_to_prefix(ode)
x1_prefix = env_1.clean_prefix(ode_prefix)

x1 = torch.LongTensor(
    [env_1.eos_index] +
    [env_1.word2id[w] for w in x1_prefix] +
    [env_1.eos_index]
).view(-1, 1)
len1 = torch.LongTensor([len(x1)])
x1, len1 = to_cuda(x1, len1)

with torch.no_grad():
    encoded_1 = encoder_1('fwd', x=x1, lengths=len1, causal=False).transpose(0, 1)

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [ ]:
beam_size = 10
with torch.no_grad():
    _, _, beam = decoder_1.generate_beam(encoded_1, len1, beam_size=beam_size, length_penalty=1.0, early_stopping=1, max_len=200)
    assert len(beam) == 1
hypotheses = beam[0].hyp
assert len(hypotheses) == beam_size

In [ ]:
print("Input function ODE:\n")
display(ode)
print("")

firstCorrect = None
for score, sent in sorted(hypotheses, key=lambda x: x[0], reverse=True):

    # parse decoded hypothesis
    ids = sent[1:].tolist()                  # decoded token IDs
    tok = [env_1.id2word[wid] for wid in ids]  # convert to prefix

    try:
        hyp = env_1.prefix_to_infix(tok)       # convert to infix
        hyp = env_1.infix_to_sympy(hyp)        # convert to SymPy

        # check whether we recover f if we differentiate the hypothesis
        res = "OK" if simplify(simplify(ode.subs(f(x), hyp), seconds=1), seconds=1) == 0 else "NO"
        if (firstCorrect==None and res=="OK"):
            firstCorrect = hyp

    except:
        res = "INVALID PREFIX EXPRESSION"
        hyp = tok

    # print result
    print("%.5f  %s  %s" % (score, res, hyp))
        
print("\nThe well-predicted expression with the highest score is:\n")
display(firstCorrect)

Input function ODE:



-f(x) + Derivative(f(x), x)


-0.44210  OK  (a8*x + x)*exp(x + 1)/x
-0.44287  OK  (a8*x + x)*exp(x + 5)/x
-0.44395  OK  (a8*x + x)*exp(x + 2)/x
-0.44656  OK  (a8*x - x)*exp(x)/x
-0.44734  OK  (a8*x + x)*exp(x + 4)/x
-0.44989  OK  (a8*x + x)*exp(x + 3)/x
-0.46156  OK  (a8*x + 2*x)*exp(x)/x
-0.50374  OK  a8*exp(x + pi/4)
-0.55816  OK  a8*exp(x + pi/2)
-0.57683  OK  (a8*x + x)*exp(x)/x

The well-predicted expression with the highest score is:



(a8*x + x)*exp(x + 1)/x

In [ ]:
from sympy.solvers import solve

def sol_with_initial_condition_1(sol, i_c):
  sol=sp.simplify(sol)
  I = sol.subs(x,0)
  i = sp.simplify(solve(I-i_c,a8)[0])
  Phi  = sol.subs(a8,i)
  return sp.simplify(Phi)

In [ ]:
sol_1 = sol_with_initial_condition_1(firstCorrect, 1)

In [ ]:
phi = sol_1.subs(x,t)

In [ ]:
display(phi)

exp(t)

In [ ]:
# trained model, e.g. "wget https://dl.fbaipublicfiles.com/SymbolicMathematics/models/fwd_bwd.pth"
model_path_2 = './fwd_bwd.pth'
assert os.path.isfile(model_path_2)

In [ ]:
params_2 = params_2 = AttrDict({

    # environment parameters
    'env_name': 'char_sp',
    'int_base': 10,
    'balanced': False,
    'positive': True,
    'precision': 10,
    'n_variables': 1,
    'n_coefficients': 0,
    'leaf_probs': '0.75,0,0.25,0',
    'max_len': 512,
    'max_int': 5,
    'max_ops': 15,
    'max_ops_G': 15,
    'clean_prefix_expr': True,
    'rewrite_functions': '',
    'tasks': 'prim_fwd',
    'operators': 'add:10,sub:3,mul:10,div:5,sqrt:4,pow2:4,pow3:2,pow4:1,pow5:1,ln:4,exp:4,sin:4,cos:4,tan:4,asin:1,acos:1,atan:1,sinh:1,cosh:1,tanh:1,asinh:1,acosh:1,atanh:1',

    # model parameters
    'cpu': False,
    'emb_dim': 1024,
    'n_enc_layers': 6,
    'n_dec_layers': 6,
    'n_heads': 8,
    'dropout': 0,
    'attention_dropout': 0,
    'sinusoidal_embeddings': False,
    'share_inout_emb': True,
    'reload_model': './fwd_bwd.pth',

})

In [ ]:
env_2 = build_env(params_2)

In [ ]:
modules_2 = build_modules(env_2, params_2)
encoder_2 = modules_2['encoder']
decoder_2 = modules_2['decoder']

In [ ]:
x = env_2.local_dict['x']

In [ ]:
C_2 = (C*1/phi).subs(t,x)

In [ ]:
f=C_2

In [ ]:
f_prefix = env_2.sympy_to_prefix(f)
print(f"f prefix: {f_prefix}")

f prefix: ['mul', 'add', 'INT+', '2', 'x', 'exp', 'mul', 'INT-', '1', 'x']


In [ ]:
x1_prefix = env_2.clean_prefix(['sub', 'derivative', 'f', 'x', 'x'] + f_prefix)
x1 = torch.LongTensor(
    [env_2.eos_index] +
    [env_2.word2id[w] for w in x1_prefix] +
    [env_2.eos_index]
).view(-1, 1)
len1 = torch.LongTensor([len(x1)])
x1, len1 = to_cuda(x1, len1)

with torch.no_grad():
    encoded = encoder_2('fwd', x=x1, lengths=len1, causal=False).transpose(0, 1)

In [ ]:
beam_size = 10
with torch.no_grad():
    _, _, beam = decoder_2.generate_beam(encoded, len1, beam_size=beam_size, length_penalty=1.0, early_stopping=1, max_len=200)
    assert len(beam) == 1
hypotheses = beam[0].hyp
assert len(hypotheses) == beam_size

In [ ]:
print(f"Input function f: {f}")
print("")

firstCorrect = None
for score, sent in sorted(hypotheses, key=lambda x: x[0], reverse=True):

    # parse decoded hypothesis
    ids = sent[1:].tolist()                  # decoded token IDs
    tok = [env_2.id2word[wid] for wid in ids]  # convert to prefix

    try:
        hyp = env_2.prefix_to_infix(tok)       # convert to infix
        hyp = env_2.infix_to_sympy(hyp)        # convert to SymPy

        # check whether we recover f if we differentiate the hypothesis
        res = "OK" if simplify(hyp.diff(x) - f, seconds=1) == 0 else "NO"
        if (firstCorrect==None and res=="OK"):
            firstCorrect = hyp

    except:
        res = "INVALID PREFIX EXPRESSION"
        hyp = tok

    # print result
    print("%.5f  %s  %s" % (score, res, hyp))
        
print("\nThe well-predicted expression with the highest score is:\n")
display(firstCorrect)

Input function f: (x + 2)*exp(-x)

-0.04285  OK  (-x - 3)*exp(-x)
-0.09961  OK  -(x + 3)*exp(-x)
-0.15323  OK  (-x**2 - 3*x)*exp(-x)/x
-0.16062  OK  (-x*(x + 4) + x)*exp(-x)/x
-0.16993  OK  (-x*(x + 3)*exp(-x) + x)/x
-0.17465  OK  (x - (x**2 + 3*x)*exp(-x))/x
-0.20028  OK  -(x**2 + 3*x)*exp(-x)/x
-0.21317  OK  (-x*(x + 3)*exp(-x) + 2*x)/x
-0.21528  OK  (-x*(x + 5) + 2*x)*exp(-x)/x
-0.31275  OK  (-x + exp(x) - 3)*exp(-x)

The well-predicted expression with the highest score is:



(-x - 3)*exp(-x)

In [ ]:
def sol_with_initial_condition_2(sol, i_c ):
  sol=sp.simplify(sol)
  I = sol.subs(x,i_c)
  Phi  = sol - I
  return sp.simplify(Phi)

In [ ]:
sol_2 = sol_with_initial_condition_2(firstCorrect, 0 )

In [ ]:
Int = sol_2.subs(x,t)

In [ ]:
Int

(-t + 3*exp(t) - 3)*exp(-t)

In [ ]:
#[Phi, Int \phi^{-1}*C]

display(phi, Int)

exp(t)

(-t + 3*exp(t) - 3)*exp(-t)